# Exploratory Analysis
### The following code is for exploratory analysis of Capital Metro Austin vehicle location data.

In [24]:
# import useful libraries
import os
import glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from dateutil.relativedelta import relativedelta as rdelta
import contextily as ctx
import hvplot.pandas
import warnings

In [66]:
# Pull data from csv files

path = r'../00_Source_Data/Vehicle_Location_History' # use your path
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

with warnings.catch_warnings():
    warnings.simplefilter('ignore') # ignore dataframe generation warnings
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [68]:
frame.columns

Index(['blockassignmentinfo', 'blockid', 'blockmthd', 'directionid',
       'distancealongstoppath', 'driver', 'gtfsstopsequence', 'heading',
       'headsign', 'headwaysecs', 'id', 'ispredictable', 'lat', 'lon',
       'nextstopid', 'nextstopname', 'previousvehicleid',
       'previousvehicleschadhsecs', 'previousvehicleschadhstr', 'routeid',
       'routename', 'routeshortname', 'schadhsecs', 'schadhstr',
       'scheduledheadwaysecs', 'servicedate', 'serviceid', 'servicename',
       'speed', 'stoppathindex', 'timecentral', 'timestamp', 'timeutc',
       'tripid', 'trippattern', 'vehicletype', 'year', 'month', '_Start_Date',
       '_End_Date'],
      dtype='object')

In [76]:
frame[['serviceid']].value_counts()

serviceid      
1-153_MRG_1        3094741
4-153_MRG_1         494169
5-153_MRG_1         441311
5001-153_MRG_1      207331
39101-153            25799
55004-153_MRG_1      15288
39201-153             8300
4-153_MRG_3           5750
26201-153             2410
26104-153             2089
40801-153             1157
14001-153              578
14101-153              509
15504-153              353
dtype: int64

Not sure if these service ids are unique to the vehicle. \
There is some confusion about 4-153_MRG_1 and 4-153_MRG_3 as these may be the same bus.\
Would need to clarify definitions with Michael Long.

In [83]:
frame[['routeshortname']].value_counts()

routeshortname
801.0             324229
10.0              275967
803.0             263498
7.0               239905
20.0              229785
                   ...  
491.0               1976
493.0               1157
490.0               1087
214.0                372
990.0                184
Length: 76, dtype: int64

In [93]:
print(frame[['previousvehicleschadhsecs']].max()[0],',', frame[['previousvehicleschadhsecs']].min()[0])

5387.0 , -14412.0


In [94]:
print(frame[['scheduledheadwaysecs']].max()[0],',', frame[['scheduledheadwaysecs']].min()[0])

57840.0 , 12.0
